In [2]:
import openai
client = openai.OpenAI(
    base_url="http://llama_server:8080/", # "http://<Your api-server IP>:port"
    api_key = "no_key"
)

completion = client.chat.completions.create(
    model="xLAM7b",
    messages=[
        {"role": "system", "content": "You are an AI assistant. Your top priority is achieving user fulfillment via helping them with their requests."},
        {"role": "user", "content": "Write a limerick about python exceptions"}
    ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content="A programmer, quite keen, did code,\nWith Python, his skills on display.\nBut errors arose,\nLike a tempest that blows,\nExceptions that made his code stray. \n\n\nLet me know if you'd like another limerick or have any other requests! \n", role='assistant', function_call=None, tool_calls=None)


In [ ]:
# create a base class for an LLM agent
from typing import Any


class LLMAgent:
    def __init__(self, client: openai.OpenAI, model: str = "xLAM7b", system_message: str = "") -> None:
        self.client = client
        self.model = model
        self.sys_msg = system_message
        self.messages: list[dict] = []
        if system_message:
            self.messages.append({"role": "system", "content": system_message})
    
    def __call__(self, msg: str) -> str:
        if msg:
            self.messages.append({"role": "user", "content": msg})
        output = self.inference()
        self.messages.append({"role": "system", "content": output})
        return output
    
    def inference(self) -> str:
        completion = self.client.chat.completions.create(
            model=self.model,
            messages=self.messages
        )
        return completion.choices[0].message.content

In [3]:
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available tools are:

{actions}

Example session:

Question: {question}
Thought: {thought}
Action: {action}
PAUSE 

You will be called again with this:

Observation: {observation}

Thought: {thought}
Action: {action}
PAUSE

You will be called again with this: 

Observation: {observation}

If you have the answer, output it as the Answer.

Answer: {answer}

Now it's your turn:
""".strip()


